In [13]:
import os
import nltk
import shutil
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser

nltk.download("punkt")  # para word_tokenize
nltk.download("stopwords")

import warnings

warnings.filterwarnings("ignore", category=SyntaxWarning)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/luizfelipew/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/luizfelipew/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
documents = [
    "Machine learning é um campo da inteligência artificial que permite que computadores aprendam padrões a partir de dados.",
    "O aprendizado de máquina dá aos sistemas a capacidade de melhorar seu desempenho sem serem explicitamente programados.",
    "Em vez de seguir apenas regras fixas, o machine learning descobre relações escondidas nos dados.",
    "Esse campo combina estatística, algoritmos e poder computacional para extrair conhecimento.",
    "O objetivo é criar modelos capazes de generalizar além dos exemplos vistos no treinamento.",
    "Aplicações de machine learning vão desde recomendações de filmes até diagnósticos médicos.",
    "Os algoritmos de aprendizado de máquina transformam dados brutos em previsões úteis.",
    "Diferente de um software tradicional, o ML adapta-se conforme novos dados chegam.",
    "O aprendizado pode ser supervisionado, não supervisionado ou por reforço, dependendo do tipo de problema.",
    "Na prática, machine learning é o motor que impulsiona muitos avanços em visão computacional e processamento de linguagem natural.",
    "Mais do que encontrar padrões, o machine learning ajuda a tomar decisões baseadas em evidências.",
]

In [17]:
def preprocess(text):
    # Tokenização em palavras
    tokens = nltk.word_tokenize(text.lower())
    # word.isalnum() elimina tudo que não seja alphanumérico , tipo ruidos que nãos ervem para análise
    tokens = [word for word in tokens if word.isalnum()]
    
    stopwords = set(nltk.corpus.stopwords.words("portuguese")) - {"e", "ou", "não"}
    tokens = [word for word in tokens if word not in stopwords]
    return tokens

In [18]:
text = "Machine learning é um campo da inteligência artificial, que permite que computadores aprendam padrões a partir de dados."
preprocess(text)

['machine',
 'learning',
 'campo',
 'inteligência',
 'artificial',
 'permite',
 'computadores',
 'aprendam',
 'padrões',
 'partir',
 'dados']

In [ ]:
if os.path.exists("indexdir"):
    shutil.rmtree("indexdir")
os.mkdir("indexdir")


In [21]:
schema = Schema(title=ID(stored=True, unique=True), content=TEXT(stored=True))

index = create_in("indexdir", schema)

writer = index.writer()
for i, doc in enumerate(documents):
    writer.add_document(title=str(i), content=doc)
writer.commit()

In [29]:
query = "machine E learning"

In [30]:
def boolean_search(query, index):
    parser = QueryParser("content", schema=index.schema)
    parsed_query = parser.parse(query)
    parsed_query

    with index.searcher() as searcher:
        results = searcher.search(parsed_query)
        return ([(hit["title"], hit["content"]) for hit in results])

In [32]:
boolean_search(query, index)

[('5',
  'Aplicações de machine learning vão desde recomendações de filmes até diagnósticos médicos.'),
 ('10',
  'Mais do que encontrar padrões, o machine learning ajuda a tomar decisões baseadas em evidências.'),
 ('2',
  'Em vez de seguir apenas regras fixas, o machine learning descobre relações escondidas nos dados.'),
 ('0',
  'Machine learning é um campo da inteligência artificial que permite que computadores aprendam padrões a partir de dados.'),
 ('9',
  'Na prática, machine learning é o motor que impulsiona muitos avanços em visão computacional e processamento de linguagem natural.')]